In [3]:
from tensorflow import keras
import tensorflow as tf
import numpy as np
import os
import cv2
try:
    from imutils import paths
except:
    !pip install imutils --quiet
    from imutils import paths
try:
    import timm
except:
    !pip install timm --quiet
    import timm
import json
import time, os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as ans
from tqdm import tqdm
import shutil
import gdown
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Model
from tensorflow.keras import layers
from tensorflow.keras.layers import *
import tensorflow_datasets as tfds
from tensorflow.keras.initializers import RandomNormal
from tensorflow.keras import *
from datetime import datetime
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.activations import sigmoid
from tensorflow.keras.layers import Dense, Input, UpSampling2D, Conv2DTranspose, Conv2D, add, Add,\
                    Lambda, Concatenate, AveragePooling2D, BatchNormalization, GlobalAveragePooling2D, \
                    Add, LayerNormalization, Activation, LeakyReLU, SeparableConv2D, Softmax, MaxPooling2D, Dropout




In [2]:
!rm -r CaiT1 CaiT3 CaiT4 CaiT5
!git clone https://github.com/shilu10/CaiT-TF2.git
!mv CaiT-TF2 CaiT12

rm: cannot remove 'CaiT1': No such file or directory
rm: cannot remove 'CaiT3': No such file or directory
rm: cannot remove 'CaiT4': No such file or directory
rm: cannot remove 'CaiT5': No such file or directory
Cloning into 'CaiT-TF2'...
remote: Enumerating objects: 250, done.
remote: Counting objects: 100% (10/10), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 250 (delta 3), reused 4 (delta 0), pack-reused 240
Receiving objects: 100% (250/250), 49.88 KiB | 2.00 MiB/s, done.
Resolving deltas: 100% (134/134), done.


In [3]:
# imagenet1k evaluation dataset path
!git clone https://github.com/EliSchwartz/imagenet-sample-images.git

Cloning into 'imagenet-sample-images'...
remote: Enumerating objects: 1012, done.
remote: Counting objects: 100% (10/10), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 1012 (delta 3), reused 5 (delta 2), pack-reused 1002
Receiving objects: 100% (1012/1012), 103.84 MiB | 16.87 MiB/s, done.
Resolving deltas: 100% (3/3), done.


In [4]:
# to port weights from pretrained pytorch to tensorflow model
#from CaiT.port_all import main
#main()


# download cait models from gdrive
url = "https://drive.google.com/uc?id=1vwKgaJf4VqCbKkUHsOIVOE2HBHUFv05I"
gdown.download(url)

Downloading...
From: https://drive.google.com/uc?id=1vwKgaJf4VqCbKkUHsOIVOE2HBHUFv05I
To: /content/cait_models.zip
100%|██████████| 3.27G/3.27G [00:34<00:00, 94.0MB/s]


'cait_models.zip'

In [ ]:
!unzip cait_models.zip

In [5]:
with open("/content/CaiT12/imagenet1k_eval/imagenet_labels.json", "r") as file:
  class_labels = json.load(file)

In [6]:
def make_log_file(fpath, model_variant, top1_accuracy, top5_accuracy):
  with open(f"{fpath}", "w") as file:
    file.write("model_variant, top1_accuracy, top5_accuracy\n")
    file.write(f"{model_variant}, {top1_accuracy}, {top5_accuracy}")

def make_log_file_all_variant(fpath):
  with open(fpath, "w") as file:
    file.write("model_variant, top1_accuracy, top5_accuracy\n")

def add_log_all_variant(fpath, model_variant, top1_accuracy, top5_accuracy):
  with open(fpath, "a") as file:
    file.write(f"{model_variant}, {top1_accuracy}, {top5_accuracy}\n")

In [7]:
def get_reverse_class_labels(class_labels):
  reverse_class_labels_dict = {}
  for key, value in class_labels.items():
    for cls in value.split(","):
      reverse_class_labels_dict[cls.strip()] = int(key)

  return reverse_class_labels_dict

In [8]:
def preprocess_data(image, label):
  #raw = tf.io.read_file(image_path)
  #image = tf.io.decode_image(raw, expand_animations = False)
  #image = tf.io.decode_jpeg(raw)
  image = tf.image.convert_image_dtype(image, dtype=tf.float32)
  #image = tf.image.resize(image, (224, 224))
  return image, label

In [9]:
def get_encoded_labels(image_files, reverse_class_labels_dict):
  test_class_labels = []

  for image_file in image_files:
    class_name = image_file.split('/')[1][10: ].split(".")[0]
    class_name = class_name.replace('_', " ")
    class_label = reverse_class_labels_dict[class_name]
    test_class_labels.append(class_label)

  test_class_labels = np.array(test_class_labels)

  return test_class_labels

In [10]:
import cv2

def create_image_arr(image_files, img_size=224):
  images = []
  for image_path in image_files:
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    if len(img.shape) != 3:
      break

    img = cv2.resize(img, (img_size, img_size))
    images.append(img)

  images = np.array(images)

  return images


In [11]:
def create_tf_dataset(images, labels, batch_size=5):
  test_ds = tf.data.Dataset.from_tensor_slices((images, labels))
  test_ds = test_ds.map(lambda x, y: preprocess_data(x, y))
  test_ds = test_ds.batch(batch_size, drop_remainder=True)
  test_ds = test_ds.prefetch(tf.data.AUTOTUNE)

  return test_ds

In [12]:
def make_prediction(model, test_ds):

  acc_obj_func = tf.keras.metrics.SparseCategoricalAccuracy(name="sparse_categorical_accuracy")
  top1_acc_obj_func = tf.keras.metrics.SparseTopKCategoricalAccuracy(k=1, name="top1_sparse_categorical_accuracy")
  top5_acc_obj_func = tf.keras.metrics.SparseTopKCategoricalAccuracy(k=5, name="top5_sparse_categorical_accuracy")

  for i, (img_batch, label_batch) in tqdm(enumerate(test_ds), total=len(test_ds)):
    prediction = model.predict(img_batch, verbose=False)

    acc_obj_func.update_state(label_batch, prediction[0])
    top1_acc_obj_func.update_state(label_batch, prediction[0])
    top5_acc_obj_func.update_state(label_batch, prediction[0])

  return acc_obj_func.result().numpy(), top1_acc_obj_func.result().numpy(), top5_acc_obj_func.result().numpy()


In [13]:
reverse_class_labels = get_reverse_class_labels(class_labels)

In [14]:
images_file_path = list(paths.list_images("imagenet-sample-images"))

In [15]:
encoded_test_labels = get_encoded_labels(images_file_path, reverse_class_labels)

In [ ]:
images_arr = create_image_arr(images_file_path, img_size=224)

In [ ]:
test_ds = create_tf_dataset(images_arr, encoded_test_labels, batch_size=5)
test_ds

<_PrefetchDataset element_spec=(TensorSpec(shape=(5, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(5,), dtype=tf.int64, name=None))>

In [ ]:
# loading model variant --> cait xxs24_224
model = keras.models.load_model("/content/models/cait_xxs24_224", compile=False)

In [ ]:
acc, top1_acc, top5_acc = make_prediction(model, test_ds)

100%|██████████| 200/200 [00:51<00:00,  3.89it/s]


In [ ]:
if not os.path.exists("logs/"):
  os.mkdir("logs")

In [ ]:
top1_acc_ = round(top1_acc*100, 2)
top5_acc_ = round(top5_acc*100, 2)

In [ ]:
make_log_file("logs/cait_xxs24_224.csv",
              "cait_xxs24_224",
              top1_accuracy=top1_acc_,
              top5_accuracy=top5_acc_)

In [ ]:
make_log_file_all_variant("logs/cait_benchmark.csv")

In [ ]:
add_log_all_variant("logs/cait_benchmark.csv", "cait_xxs24_224", top1_acc_, top5_acc_)

In [ ]:
print(f"Model Variant: cait_xxs24_224, top1_accuracy: {top1_acc_}, top5_accuracy: {top5_acc_}")

Model Variant: cait_xxs24_224, top1_accuracy: 82.5, top5_accuracy: 95.0


In [ ]:
# model variant cait_xxs36_224
model = keras.models.load_model("/content/models/cait_xxs36_224", compile=False)

In [ ]:
acc, top1_acc, top5_acc = make_prediction(model, test_ds)

100%|██████████| 200/200 [01:06<00:00,  3.00it/s]


In [ ]:
top1_acc_ = round(top1_acc*100, 2)
top5_acc_ = round(top5_acc*100, 2)

In [ ]:
make_log_file("logs/cait_xxs36_224.csv",
              "cait_xxs36_224",
              top1_accuracy=top1_acc_,
              top5_accuracy=top5_acc_)

In [ ]:
add_log_all_variant("logs/cait_benchmark.csv", "cait_xxs36_224", top1_acc_, top5_acc_)

In [ ]:
print(f"Model Variant: cait_xxs36_224, top1_accuracy: {top1_acc_}, top5_accuracy: {top5_acc_}")

Model Variant: cait_xxs36_224, top1_accuracy: 84.5, top5_accuracy: 96.1


In [ ]:
with open("logs/cait_variants.csv", "a") as file:
  file.write(f"cait_xxs36_224, top1_acc_, top5_acc_\n")

In [ ]:
!cp -r /content/logs /content/drive/MyDrive

In [ ]:
# model variant cait_s24_224
model = keras.models.load_model("/content/models/cait_s24_224", compile=False)

In [ ]:
acc, top1_acc, top5_acc = make_prediction(model, test_ds)

100%|██████████| 200/200 [01:04<00:00,  3.09it/s]


In [ ]:
top1_acc_ = round(top1_acc*100, 2)
top5_acc_ = round(top5_acc*100, 2)

In [ ]:
make_log_file("logs/cait_s24_224.csv",
              "cait_s24_224",
              top1_accuracy=top1_acc_,
              top5_accuracy=top5_acc_)

In [ ]:
add_log_all_variant("logs/cait_benchmark.csv", "cait_s24_224", top1_acc_, top5_acc_)

In [ ]:
print(f"Model Variant: cait_s24_224, top1_accuracy: {top1_acc_}, top5_accuracy: {top5_acc_}")

Model Variant: cait_s24_224, top1_accuracy: 89.8, top5_accuracy: 98.4


In [ ]:
# models with 384 image size

images_arr = create_image_arr(images_file_path, img_size=384)


In [ ]:
test_ds = create_tf_dataset(images_arr, encoded_test_labels, batch_size=2)
test_ds

<_PrefetchDataset element_spec=(TensorSpec(shape=(2, 384, 384, 3), dtype=tf.float32, name=None), TensorSpec(shape=(2,), dtype=tf.int64, name=None))>

In [ ]:
# model variant cait_s24_224
model = keras.models.load_model("/content/models/cait_xxs24_384", compile=False)

In [ ]:
acc, top1_acc, top5_acc = make_prediction(model, test_ds)

100%|██████████| 1000/1000 [04:49<00:00,  3.46it/s]


In [ ]:
top1_acc_ = round(top1_acc*100, 2)
top5_acc_ = round(top5_acc*100, 2)

In [ ]:
make_log_file("logs/cait_xxs24_384.csv",
              "cait_xxs24_384",
              top1_accuracy=top1_acc_,
              top5_accuracy=top5_acc_)

In [ ]:
add_log_all_variant("logs/cait_benchmark.csv", "cait_xxs24_384", top1_acc_, top5_acc_)

In [ ]:
print(f"Model Variant: cait_xxs24_384, top1_accuracy: {top1_acc_}, top5_accuracy: {top5_acc_}")

Model Variant: cait_xxs24_384, top1_accuracy: 85.5, top5_accuracy: 96.5


In [ ]:
!cp -r /content/logs /content/drive/MyDrive

In [ ]:
# model variant cait_s24_224
model = keras.models.load_model("/content/models/cait_xxs36_384", compile=False)

In [ ]:
acc, top1_acc, top5_acc = make_prediction(model, test_ds)

In [ ]:
top1_acc_ = round(top1_acc*100, 2)
top5_acc_ = round(top5_acc*100, 2)

In [ ]:
make_log_file("logs/cait_xxs36_384.csv",
              "cait_xxs36_384",
              top1_accuracy=top1_acc_,
              top5_accuracy=top5_acc_)

In [ ]:
add_log_all_variant("logs/cait_benchmark.csv", "cait_xxs36_384", top1_acc_, top5_acc_)

In [ ]:
print(f"Model Variant: cait_xxs36_384, top1_accuracy: {top1_acc_}, top5_accuracy: {top5_acc_}")

Model Variant: cait_xxs36_384, top1_accuracy: 86.2, top5_accuracy: 97.5


In [ ]:
!cp -r /content/logs /content/drive/MyDrive

In [ ]:
# model variant cait_xs24_384
model = keras.models.load_model("/content/models/cait_xs24_384", compile=False)

In [ ]:
acc, top1_acc, top5_acc = make_prediction(model, test_ds)

100%|██████████| 500/500 [04:12<00:00,  1.98it/s]


In [ ]:
top1_acc_ = round(top1_acc*100, 2)
top5_acc_ = round(top5_acc*100, 2)

In [ ]:
make_log_file("logs/cait_xs24_384.csv",
              "cait_xs24_384",
              top1_accuracy=top1_acc_,
              top5_accuracy=top5_acc_)

In [ ]:
add_log_all_variant("logs/cait_benchmark.csv", "cait_xs24_384", top1_acc_, top5_acc_)

In [ ]:
print(f"Model Variant: cait_xs24_384, top1_accuracy: {top1_acc_}, top5_accuracy: {top5_acc_}")

Model Variant: cait_xs24_384, top1_accuracy: 86.2, top5_accuracy: 97.5


In [ ]:
!cp -r /content/logs /content/drive/MyDrive

In [ ]:
# model variant cait_s24_384
model = keras.models.load_model("/content/models/cait_s24_384", compile=False)

In [ ]:
acc, top1_acc, top5_acc = make_prediction(model, test_ds)

100%|██████████| 500/500 [04:59<00:00,  1.67it/s]


In [ ]:
top1_acc_ = round(top1_acc*100, 2)
top5_acc_ = round(top5_acc*100, 2)

In [ ]:
make_log_file("logs/cait_s24_384.csv",
              "cait_s24_384",
              top1_accuracy=top1_acc_,
              top5_accuracy=top5_acc_)

In [ ]:
add_log_all_variant("logs/cait_benchmark.csv", "cait_s24_384", top1_acc_, top5_acc_)

In [ ]:
print(f"Model Variant: cait_s24_384, top1_accuracy: {top1_acc_}, top5_accuracy: {top5_acc_}")

Model Variant: cait_s24_384, top1_accuracy: 91.3, top5_accuracy: 98.6


In [ ]:
!cp -r /content/logs /content/drive/MyDrive

In [ ]:
# model variant cait_s24_384
model = keras.models.load_model("/content/models/cait_s36_384", compile=False)

In [ ]:
acc, top1_acc, top5_acc = make_prediction(model, test_ds)

100%|██████████| 500/500 [06:49<00:00,  1.22it/s]


In [ ]:
top1_acc_ = round(top1_acc*100, 2)
top5_acc_ = round(top5_acc*100, 2)

In [ ]:
make_log_file("logs/cait_s36_384.csv",
              "cait_s36_384",
              top1_accuracy=top1_acc_,
              top5_accuracy=top5_acc_)

In [ ]:
add_log_all_variant("logs/cait_benchmark.csv", "cait_s36_384", top1_acc_, top5_acc_)

In [ ]:
print(f"Model Variant: cait_s36_384, top1_accuracy: {top1_acc_}, top5_accuracy: {top5_acc_}")

Model Variant: cait_s36_384, top1_accuracy: 91.3, top5_accuracy: 98.7


In [ ]:
!cp -r /content/logs /content/drive/MyDrive

In [ ]:
# model variant cait_s24_384
model = keras.models.load_model("/content/models/cait_m36_384", compile=False)

In [ ]:
acc, top1_acc, top5_acc = make_prediction(model, test_ds)

100%|██████████| 500/500 [39:37<00:00,  4.75s/it]


In [ ]:
top1_acc_ = round(top1_acc*100, 2)
top5_acc_ = round(top5_acc*100, 2)

In [ ]:
make_log_file("logs/cait_m36_384.csv",
              "cait_m36_384",
              top1_accuracy=top1_acc_,
              top5_accuracy=top5_acc_)

In [ ]:
add_log_all_variant("logs/cait_benchmark.csv", "cait_m36_384", top1_acc_, top5_acc_)

In [ ]:
print(f"Model Variant: cait_m36_384, top1_accuracy: {top1_acc_}, top5_accuracy: {top5_acc_}")

Model Variant: cait_m36_384, top1_accuracy: 92.9, top5_accuracy: 99.4


In [ ]:
!cp -r /content/logs /content/drive/MyDrive

In [16]:
# models with image size of 448
images_arr = create_image_arr(images_file_path, img_size=448)
test_ds = create_tf_dataset(images_arr, encoded_test_labels, batch_size=2)
test_ds

<_PrefetchDataset element_spec=(TensorSpec(shape=(2, 448, 448, 3), dtype=tf.float32, name=None), TensorSpec(shape=(2,), dtype=tf.int64, name=None))>

In [20]:
!cp -r /content/content/drive/MyDrive/models .

In [4]:
# model variant cait_m48_448
model = keras.models.load_model("/content/models/cait_m48_448", compile=False)

In [ ]:
acc, top1_acc, top5_acc = make_prediction(model, test_ds)

  0%|          | 0/500 [00:00<?, ?it/s]

In [ ]:
top1_acc_ = round(top1_acc*100, 2)
top5_acc_ = round(top5_acc*100, 2)

In [ ]:
make_log_file("logs/cait_m48_448.csv",
              "cait_m48_448",
              top1_accuracy=top1_acc_,
              top5_accuracy=top5_acc_)

In [ ]:
add_log_all_variant("logs/cait_benchmark.csv", "cait_m48_448", top1_acc_, top5_acc_)

In [ ]:
print(f"Model Variant: cait_m48_448, top1_accuracy: {top1_acc_}, top5_accuracy: {top5_acc_}")

In [ ]:
!cp -r /content/logs /content/drive/MyDrive

In [2]:
!cp -r drive/MyDrive/logs .